# **ch.05 트리 알고리즘**  
## 05-3 트리의 앙상블  
### 풀어야 할 문제:  
> ### structured data에 대체적으로 좋은 performance를 보여주는 machine-learning algorithm을 찾으라  
### *정형 데이터와 비정형 데이터*  
structured data: 구조화되어 csv, database, excel 등의 tool로 저장하거나 다루기 쉬운 data  
&rarr; 지금까지 다루었던 data는 structured data.


unstructured data: 구조화되지 않아 앞선 tool로 저장하거나 표현하기 어려운 data  
&rarr; photo, music, text 등


structured data를 다루는 데 가장 뛰어난 성과를 내는 algorithm이 ensemble learning임  
ensemble learning algorithm들은 대부분 decision tree를 기반으로 함  
이번에 다룰 algorithm들이 모두 ensemble algorithm


### *랜덤 포레스트*  
random forest (scikit-learn)
+ 소개
    + ensemble learning의 대표 주자 중 하나
    + stable performance로 널리 사용됨
    + ensemble learning을 적용할 때 가장 먼저 random forest를 시도해 보기를 권장
+ 동작
    + decision tree를 random하게 만들어 decision tree의 forest를 만들고, 각 tree의 prediction을 사용해 최종으로 predict
+ data
    + each tree를 training하기 위한 data를 random하게 만듦
        + 입력한 train data에서 random하게 sample을 추출하는데, 중복을 허용 (bootstrap sample)
        + 일반적으로 이러한 bootstrap sample은 train set과 같은 크기로 만듦
+ node 분할
    + entire feature 중, 일부 feature를 random하게 고르고 최선의 분할을 선택
    + RandomForesetClassifier는 entire feature의 수의 route만큼 feature 선택
    + RandomForesetRegressor는 entire feature 사용
+ rnadom forest는 기본적으로 100개의 decision tree를 이런 방식으로 training
+ classification일 때 tree의 class별 probability를 평균하여 가장 높은 probability를 가진 class를 prediction으로 삼음
+ regression일 때 each tree의 prediction을 평균함  


random forest는 random하게 선택한 sample과 feature를 사용하기 때문에 overfiting을 방지해주고, validation set과 test set에서 stable한 performance를 보여줌


RandomForestClassifier class를 wine classification 문제에 적용하기  
+ data 준비하기
    + data 불러오기
    + train set, test set 나누기 

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

+ cross validation   


train set에 대한 score도 출력할 지 여부를 지정하는 return_train_score parameter를 True로 지정

In [4]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)

scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


다소 overfitting  
(여기서는 해결하지 않음. algorithm을 알아보는 것이 목적이므로)


random forest는 decision tree의 ensemble이기에 decision tree class가 제공하는 대부분의 중요한 parameter를 제공  
또한 decision tree와 마찬가지로 featrue importance를 제공  
random forest의 feature importance는 each tree의 feature importance를 취합한 것


random forest의 feature imoprtance 확인

In [7]:
rf.fit(train_input, train_target)

print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


앞선 decision tree의 feature importance와 비교할 때, 어느 한 feature의 importance가 지나치게 크지 않음  
$\because$ random forest가 feature의 일부를 random하게 선택하여 decision tree를 training하기 때문  
&rarr; 한 가지 feature에 과도하게 집중하지 않고 좀 더 많은 features가 training에 기여할 기회를 얻음  
&rarr; overfitting을 억제하고 general performance를 높여줌


random forest는 자체적으로 model을 평가할 수 있음  
$\because$ bootstrap sample을 사용하기 때문에, bootstrap sample에 포함되지 않은 data sample(OOB(out of bag) sample)이 발생  
&rarr; validation set의 역할을 하는 oob sample을 사용하여 model을 평가할 수 있음  
oob score를 활용하면 cross validation을 대신할 수 있어 train set에 더 많은 data를 사용할 수 있음  
이 score를 얻으려면 oob_score parameter를 True로 지정해야 함


oob score 확인

In [9]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)

print(rf.oob_score_)

0.8934000384837406


### *엑스트라 트리*  
extra trees
+ 소개
    + random forest와 매우 비슷하게 동작
        + 100개의 decision tree를 training
        + decision tree가 제공하는 대부분의 parameter를 지원
        + feature 중, 일부를 random하게 선택해 node를 분할하는 데 사용
    + random forest와의 차이점
        + bootstrap sample을 사용하지 않고, entire train set을 사용 
        + node 분할 시, 최적의 분할을 찾지 않고 random으로 분할


하나의 decision tree에서 feature를 random하게 분할하면 performance가 낮아지겠지만, 많은 tree를 ensemble하기에 overfitting을 억제하고 validation set의 score를 높이는 효과가 있음


scikit-learn의 extra trees인 ExtraTreesClassifier class의 cross validation score 확인

In [10]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)

scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


random forest와 비슷한 score 
이 예제는 feature가 많지 않아 두 model의 차이가 크지 않음  
보통 extra trees의 randomness(무작위성)가 더 크므로 random forest보다 많은 tree를 training해야 하지만, random하게 node를 분할하기 때문에 계산 속도가 빠름  


extra trees도 feature importance를 제공


extra trees의 feature importance 확인

In [11]:
et.fit(train_input, train_target)

print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


random forest와 마찬가지로 특정 feature에 대한 의존성이 작음


### *그레이디언트 부스팅*  
gradient boosting  
+ 소개
    + depth가 얕은 decision tree를 사용하여 이전 tree의 error를 보완하는 방식으로 ensemble하는 방법
    + scikit-learn의 GradientBoostingClassifier는 기본적으로 depth=3의 decision tree를 100개 사용
    + depth가 얕기 때문에 overfitting을 잘 억제하고 높은 general performance를 기대할 수 있음
+ 동작
    + gradient descent를 사용하여 tree를 ensemble에추가
    + classification에서는 logistic error function(bainary cross entropy)를 사용
    + regression에서는 MSE(mean squared error) function을 사용
    + gradient descent에서 조금씩 이동해야 했듯이, gradient boosting에서는 얕은 tree를 활용


GradientBoostingClassifier class의 cross validation score 확인

In [12]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)

scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


overfitting을 잘 억제함  
gradient boosting은 tree의 개수를 늘려도 overfitting에 매우 강함  
learning rate를 증가시키고, tree 개수를 늘리면 조금 더 performance 향상을 기대할 수 있음


tree 개수(n_estimators)를 500, learning rate(learning_rate)를 0.2로 다시 cross validation 수행

In [14]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)

scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


gradient boosting도 feature importance 제공


feature importance 확인

In [15]:
gb.fit(train_input, train_target)

print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


random forest보다 특정 feature에 대한 의존도가 높음


tree training에 사용할 train set의 비율을 정하는 subsample parameter의 값을 지정하여 일부 sample을 선택하여 training할 수 있음  
이는 stochastic gradient descent나 minibatch gradient descent와 비슷함


일반적으로 gradient boosting이 random forest보다 조금 더 높은 performance를 보여줌  
그러나 gradient boosting은 순서대로 tree를 추가하기 때문에 training 속도가 느림 (병렬 계산 불가)


### *히스토그램 기반 그레이디언트 부스팅*  
histogram-based gradient boosting
+ 소개
    + gradient boosing의 약점인 tarining 속도를 개선하고 performance를 개선한 model
    + strctured data를 다루는 machine-learning algorithm 중에 가장 인기가 높은 algorithm
+ 동작
    + feature를 256개의 구간으로 나누기에 node 분할 시 최적의 분할을 매우 빠르게 찾을 수 있음
    + 256개의 feature 구간 중에서 하나를 떼어 놓고 누락된 값을 위해 사용하므로 input에 누락된 feature가 있어도 따로 preprocessing할 필요가 없음


scikit-learn의 HistGradientBoostingClassifier는 tree의 개수를 지정하는데 n_estimators parameter가 아닌 iteration 횟수를 지정하는 max_iter parameter를 사용함


HistGradinetBoostingClassifier class에 대한 cross validation score 확인

In [16]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)

scores = cross_validate(hgb, train_input, train_target, return_train_score=True)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/private/var/containers/Bundle/Application/138F3D65-EE25-402E-9D8D-6C8F0FACAC5C/Carnets-sci.app/Library/lib/python3.11/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


overfitting을 억제하면서 gradient boosting보다 조금 더 높은 performance를 보여줌


histogram-based gradient boosting의 feature importance 확인  
permutation_importance() function 사용  
이 function은 feature를 하나씩 fandom하게 섞어 model의 performance가 변화하는지를 관찰하여 어떤 feature가 중요한지 계산  
n_repeats parameter는 random하게 섞을 횟수를 지정  
train set 뿐 아니라, test set에도 적용할 수 있음


model을 training하고 train set에 대한 feature importance 확인하기

In [17]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)

result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)

print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


permutation_importance() function이 return하는 object는 importace, importance의 mean(importances_mean), importance의 std(importances_std)를 담고 있음


output을 보면 random forest와 비슷한 ratio임을 알 수 있음


test set에서 importance 계산

In [18]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)

print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


test set에서는 train set에서보다 조금 더 sugar에 의존하는 것을 알 수 있음  
&rarr; 이러한 분석을 통해 model이 실전에 투입되었을 때, 어떤 feature에 관심을 둘 지 알 수 있음


최종으로 histogram-based gradient boosting을 사용해 test set에서의 performance 확인

In [19]:
hgb.score(test_input, test_target)

0.8723076923076923

ensemble model은 확실히 decision tree보다 좋은 performance를 보여줌